In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import confusion_matrix, classification_report, f1_score, accuracy_score,recall_score,precision_score
from sklearn.model_selection import train_test_split


import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import cv2
import optuna


data_dir = 'images'
images = os.listdir(data_dir)


test_images = pd.read_csv('CXR8/test_list.txt', header=None)


test_images

train_images = pd.read_csv('CXR8/train_val_list.txt', header=None)

train_images

labels = pd.read_csv('CXR8/Data_Entry_2017_v2020.csv')



all_labels = '|'.join(labels['Finding Labels'].unique())
all_labels = all_labels.split('|')
all_labels = list(set(all_labels))



for label in all_labels:
    labels[label] = labels['Finding Labels'].apply(lambda x: 1 if label in x else 0)


labels


tenso = torch.tensor(labels[all_labels].values).float()
data = pd.DataFrame()
data['Image Index'] = labels['Image Index']
data[all_labels] = tenso


data = data.drop(columns=['No Finding'])


all_labels

all_labels_without_no_finding = all_labels.copy()
all_labels_without_no_finding.remove('No Finding')
all_labels_without_no_finding

from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image

torch.cuda.empty_cache()
class CXR8Dataset(Dataset):
    def __init__(self, data, data_dir, transform=None):
        super().__init__()
        self.data = data
        self.data_dir = data_dir
        self.transform = transform

    def __len__(self):
        return len(self.data)

    def __getitem__(self, index):
        img_name = self.data.iloc[index, 0]
        img_path = os.path.join(self.data_dir, img_name)
        img = Image.open(img_path)
        img = np.array(img)
   
        if len(img.shape) == 2:
            img = img[:, :, np.newaxis]
            img = np.concatenate([img, img, img], axis=2)
        if len(img.shape)>2:
            img = img[:,:,0]
            img = img[:, :, np.newaxis]
            img = np.concatenate([img, img, img], axis=2)

        img = Image.fromarray(img)
    

      

        if self.transform is not None:
            img = self.transform(img)
        label = self.data.iloc[index, 1:].values
        label = np.array(label, dtype=np.float32)
        label = torch.tensor(label, dtype=torch.float32)

        return img, label
# create a transform

mean = [0.485, 0.456, 0.406]
std = [0.229, 0.224, 0.225]





   



   

   


    # place the images in train_images in the train set and the images in test_images in the test set
train_val_data = data[data['Image Index'].isin(train_images[0].values)]
test_data = data[data['Image Index'].isin(test_images[0].values)]

patients_ids = pd.read_csv('CXR8/Data_Entry_2017_v2020.csv')
patients_ids = patients_ids[['Image Index', 'Patient ID']]

train_val_data = pd.merge(train_val_data, patients_ids, on='Image Index')



unique_patient_ids = train_val_data['Patient ID'].unique()

# Split patient IDs into training and validation sets
train_patient_ids, val_patient_ids = train_test_split(
    unique_patient_ids, 
    test_size=0.1, 
    random_state=42
)

# Create train and validation data based on the patient ID split
train_data = train_val_data[train_val_data['Patient ID'].isin(train_patient_ids)]
val_data = train_val_data[train_val_data['Patient ID'].isin(val_patient_ids)]

train_data = train_data.drop(columns=['Patient ID'])
val_data = val_data.drop(columns=['Patient ID'])


In [5]:
train_data = train_data.drop(columns=['Patient ID'])
val_data = val_data.drop(columns=['Patient ID'])


In [6]:
len(train_data), len(val_data), len(test_data)

(77988, 8536, 25596)

In [7]:
from torchvision.models import DenseNet121_Weights

# List all available weights
print(DenseNet121_Weights)

<enum 'DenseNet121_Weights'>
